In [2]:
# Load CSV (using python)
import csv
import numpy as np
import math

filename = 'car.csv'
raw_data = open(filename, 'rt')
reader = csv.reader(raw_data, delimiter=',', quoting=csv.QUOTE_NONE)
x = list(reader)
data = np.array(x)
print(data.shape)

data = data.astype('float')
y = data[:, 48]
print(sum(y))
X = data[:, :48]

(58509, 49)
351054.0


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

feature = len(X)
prop = 0.3 # the proportion of training samples to be extracted

def get_train_X(whole):
    train = whole[:int(feature * prop),:]
    return np.asmatrix(train[:, :48])
def get_train_y(whole):
    train = whole[:int(feature * prop),:]
    return np.squeeze(np.asarray(train[:, 48]))
def get_test_X(whole):
    test = whole[int(feature * prop):,:]
    return np.asmatrix(test[:, :48])
def get_test_y(whole):
    test = whole[int(feature * prop):,:]
    return np.squeeze(np.asarray(test[:, 48]))
    
def cross_validation(whole, fold):
    error = 0
    for i in range(fold):
        np.random.shuffle(whole)
        logistic = LogisticRegression()
        logistic.fit(get_train_X(whole), get_train_y(whole))
        error = error + (1 - logistic.score(get_test_X(whole), get_test_y(whole)))
    error = error / float(fold)
    return 1 - error
    
def main():
    print("running Logistic Regression...with training proportion of ", prop)
    #prepare training set and testing set
    np.random.shuffle(data)

    logistic = LogisticRegression()
    logistic.fit(get_train_X(data), get_train_y(data))
    acc = logistic.score(get_train_X(data), get_train_y(data))
    print ("training set size ", int(feature * prop))
    print ("training accuracy ", acc * 100, "%")
    
    acc = logistic.score(get_test_X(data), get_test_y(data))
    print ("testing set size ", int(feature * (1-prop)))
    print ("testing accuracy ", acc * 100, "%")
         
    acc = cross_val_score(logistic, X, y, cv=10).mean()
    #acc = cross_validation(data, 10)
    print ("validation accuracy ", acc * 100, "%")
    
main()

In [3]:
from sklearn import svm
from sklearn.model_selection import cross_val_score

feature = len(X)
prop = 0.4 # the proportion of training samples to be extracted

def get_train_X(whole):
    train = whole[:int(feature * prop),:]
    return np.asmatrix(train[:, :48])
def get_train_y(whole):
    train = whole[:int(feature * prop),:]
    return np.squeeze(np.asarray(train[:, 48]))
def get_test_X(whole):
    test = whole[int(feature * prop):,:]
    return np.asmatrix(test[:, :48])
def get_test_y(whole):
    test = whole[int(feature * prop):,:]
    return np.squeeze(np.asarray(test[:, 48]))
    
def cross_validation(whole, fold, C):
    error = 0
    for i in range(fold):
        np.random.shuffle(whole)
        clf = svm.SVC(kernel = 'linear', C=C)
        clf.fit(get_train_X(whole), get_train_y(whole))
        error = error + (1 - clf.score(get_test_X(whole), get_test_y(whole)))
    error = error / float(fold)
    return 1 - error
def find_C(whole):
    print("finding C...")
    C = 1
    step = 2
    optimal_C = C
    min_acc = 0
    while (C < 10):
        print("cross validating...")
        acc = cross_validation(whole, 3, C)
        print("accuracy for ", C, " is ", acc)
        if acc > min_acc:
            min_acc = acc
            optimal_C = C
        C = C + step
    print("choice of C is ", optimal_C)
    return optimal_C

def main():
    print("running SVM(rbf kernel)...with training proportion of ", prop)
    #prepare training set and testing set
    np.random.shuffle(data)
    
    C = find_C(data)
    #C = 9
    np.random.shuffle(data)
    
    print ("training set size ", int(feature * prop))
    print ("testing set size ", int(feature * (1-prop)))
#    clf = svm.SVC(kernel = 'linear', C=1)
#    clf.fit(get_train_X(data), get_train_y(data))
#    acc = clf.score(get_train_X(data), get_train_y(data))
#    print ("training accuracy with C = 1", acc * 100, "%")
#    acc = clf.score(get_test_X(data), get_test_y(data))
#    print ("testing accuracy with C = 1", acc * 100, "%")
    
    
    clf = svm.SVC(kernel = 'linear', C=C)
    clf.fit(get_train_X(data), get_train_y(data))
    acc = clf.score(get_train_X(data), get_train_y(data))   
    print ("training accuracy with C = ", C, " ", acc * 100, "%")
    acc = clf.score(get_test_X(data), get_test_y(data))
    print ("testing accuracy with C = ", C, acc * 100, "%")
    
    #acc = cross_val_score(clf, X, y, cv=10).mean()
    acc = cross_validation(data, 3, C)
    print ("validation accuracy ", acc * 100, "%")
main()

running SVM(rbf kernel)...with training proportion of  0.4
finding C...
cross validating...
accuracy for  1  is  0.829430866519
cross validating...
accuracy for  3  is  0.888176759908
cross validating...
accuracy for  5  is  0.896342505555
cross validating...
accuracy for  7  is  0.902219943409
cross validating...
accuracy for  9  is  0.903245409142
choice of C is  9
training set size  23403
testing set size  35105
training accuracy with C =  9   90.9584241337 %
testing accuracy with C =  9 90.0159516892 %
validation accuracy  90.206802256 %


In [ ]:
from sklearn.ensemble import RandomForestClassifier

feature = len(X)
prop = 0.1 # the proportion of training samples to be extracted

def get_train_X(whole):
    train = whole[:int(feature * prop),:]
    return np.asmatrix(train[:, :48])
def get_train_y(whole):
    train = whole[:int(feature * prop),:]
    return np.squeeze(np.asarray(train[:, 48]))
def get_test_X(whole):
    test = whole[int(feature * prop):,:]
    return np.asmatrix(test[:, :48])
def get_test_y(whole):
    test = whole[int(feature * prop):,:]
    return np.squeeze(np.asarray(test[:, 48]))
    
def cross_validation(whole, fold, T):
    error = 0
    for i in range(fold):
        np.random.shuffle(whole)
        clf = RandomForestClassifier(n_estimators=T)
        clf.fit(get_train_X(whole), get_train_y(whole))
        error = error + (1 - clf.score(get_test_X(whole), get_test_y(whole)))
    error = error / float(fold)
    return 1 - error

def find_T(whole):
    T = 100
    step = 10
    optimal_T = T
    min_acc = 0
    while (T < 300):
        acc = cross_validation(whole, 5, T)
        #print("accuracy for ", T, "is ", acc)
        if acc > min_acc:
            min_acc = acc
            optimal_T = T
        T = T + step
    print("choice of T is ", optimal_T)
    return optimal_T

def main():
    print("running Random Forest...with training proportion of ", prop)
    #prepare training set and testing set
    np.random.shuffle(data)
    
    #T = find_T(data)
    T = 100
    np.random.shuffle(data)
    
    
    print ("training set size ", int(feature * prop))
    print ("testing set size ", int(feature * (1-prop)))
    clf = RandomForestClassifier(n_estimators=200)
    clf.fit(get_train_X(data), get_train_y(data))
    acc = clf.score(get_train_X(data), get_train_y(data))
    print ("training accuracy with T = 200", acc * 100, "%")
    acc = clf.score(get_test_X(data), get_test_y(data))
    print ("testing accuracy with T = 200", acc * 100, "%")
    
    
    clf = RandomForestClassifier(n_estimators=T)
    clf.fit(get_train_X(data), get_train_y(data))
    acc = clf.score(get_train_X(data), get_train_y(data))   
    print ("training accuracy with T = ", T, " ", acc * 100, "%")
    acc = clf.score(get_test_X(data), get_test_y(data))
    print ("testing accuracy with T = 200", acc * 100, "%")
    
    #acc = cross_val_score(clf, X, y, cv=10).mean()
    acc = cross_validation(data, 10, T)
    print ("validation accuracy ", acc * 100, "%")
    
main()

In [ ]:
from numpy import linalg as LA
import operator

dimension = 48
feature = len(X)
prop = 0.1 # the proportion of training samples to be extracted

def get_train(whole):
    return whole[:int(feature * prop),:]
def get_test(whole):
    whole[int(feature * prop):,:]
def get_train_X(whole):
    train = whole[:int(feature * prop),:]
    return np.asmatrix(train[:, :dimension])
def get_train_y(whole):
    train = whole[:int(feature * prop),:]
    return np.squeeze(np.asarray(train[:, dimension]))
def get_test_X(whole):
    test = whole[int(feature * prop):,:]
    return np.asmatrix(test[:, :dimension])
def get_test_y(whole):
    test = whole[int(feature * prop):,:]
    return np.squeeze(np.asarray(test[:, dimension]))

def get_neighbors(train, goal_X, k):
    distances = []
    train_X = np.asmatrix(train[:, :dimension])
    for i in range(len(train_X)):
        dist = LA.norm(train_X[i] - goal_X)
        distances.append((train[i], dist))
    distances.sort(key=operator.itemgetter(1))
    
    neighbors = np.zeros((k, train.shape[1]))
    for i in range(k):
        neighbors[i] = np.copy(np.asarray(distances[i][0]))
    return neighbors

def predict(neighbors):
    count0 = 0
    count1 = 0
    for i in range(len(neighbors)):
        if(neighbors[i, dimension] == 0):
            count0 = count0 + 1
        if(neighbors[i, dimension] == 1):
            count1 = count1 + 1
    if(count0 > count1):
        return 0
    else:
        return 1

#input: testing y and neightbors
def get_error(train, test_X, test_y, k):
    result = 0
    miss = 0
    for i in range(len(test_y)):
        neighbors = get_neighbors(train, test_X[i], k)
        result = predict(neighbors)
        if result != test_y[i]:
            miss = miss + 1
    return miss/len(test_y)
       
def cross_validation(whole, fold, k):
    error = 0
    for i in range(fold):
        np.random.shuffle(whole)
        train = get_train(whole)
        test_X = get_test_X(whole)
        test_y = get_test_y(whole)
        error = error + get_error(train, test_X, test_y, k)
    error = error / float(fold)
    return error

def find_K(whole, lower, upper):
    K = lower
    optimal_K = lower
    min_error = 1
    while (K < (upper + 1)):
        error = cross_validation(whole, 10, K)
        print("error for ", K, "is ", error)
        if error < min_error:
            min_error = error
            optimal_K = K
        K = K + 2
    print("choice of K is ", optimal_K)
    return optimal_K

def main():
    print("running K-Nearest Neighbors...")
    #prepare training set and testing set
    whole = data
    np.random.shuffle(whole)
    K = find_K(whole, 1, 7)
    
    np.random.shuffle(whole)

    error = get_error(get_train(whole), get_train_X(whole), get_train_y(whole), K)
    print ("training error ", error)
    
    error = get_error(get_train(whole), get_test_X(whole), get_test_y(whole), K)
    print ("testing error ", error)
    
    error = cross_validation(whole, 10, K)
    print ("validation error ", error)
    
main()

running K-Nearest Neighbors...


In [5]:
from sklearn import tree

dimension = 48
feature = len(X)
prop = 0.4 # the proportion of training samples to be extracted

def get_train_X(whole, prop):
    train = whole[:int(feature * prop),:]
    return np.asmatrix(train[:, :dimension])
def get_train_y(whole, prop):
    train = whole[:int(feature * prop),:]
    return np.squeeze(np.asarray(train[:, dimension]))
def get_test_X(whole, prop):
    test = whole[int(feature * prop):,:]
    return np.asmatrix(test[:, :dimension])
def get_test_y(whole, prop):
    test = whole[int(feature * prop):,:]
    return np.squeeze(np.asarray(test[:, dimension]))
    
def cross_validation(whole, fold, D):
    error = 0
    for num2 in range(0, fold):
        np.random.shuffle(whole)
        clf = tree.DecisionTreeClassifier(max_depth = D).fit(X = get_train_X(whole, prop), y = get_train_y(whole, prop))
        error = error + 1 - clf.score(get_test_X(whole, prop), get_test_y(whole, prop))
    error = error / float(fold)
    return error



def main():
    whole = data
    #prepare training set and testing set
    np.random.shuffle(whole)
    
    print("running decision tree...with proportion of training", prop)
    #errors
    np.random.shuffle(whole)
    clf = tree.DecisionTreeClassifier(max_depth = 16).fit(X = get_train_X(whole, prop), y = get_train_y(whole, prop))
    acc = clf.score(get_train_X(whole, prop), get_train_y(whole, prop))
    print ("training accuracy ", acc * 100, "%")
    
    acc = clf.score(get_test_X(whole, prop), get_test_y(whole, prop))
    print ("testing accuracy ", acc * 100, "%")
    
    error = cross_validation(whole, 10, 16)
    print ("validation accuracy ", acc * 100, "%")
    
main()

running decision tree...with proportion of training 0.4
training accuracy  99.7863521771 %
testing accuracy  97.7980971914 %
validation accuracy  97.7980971914 %


In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score

dimension = 48
feature = len(X)
prop = 0.4# the proportion of training samples to be extracted

def get_train_X(whole, prop):
    train = whole[:int(feature * prop),:]
    return np.asmatrix(train[:, :dimension])
def get_train_y(whole, prop):
    train = whole[:int(feature * prop),:]
    return np.squeeze(np.asarray(train[:, dimension]))
def get_test_X(whole, prop):
    test = whole[int(feature * prop):,:]
    return np.asmatrix(test[:, :dimension])
def get_test_y(whole, prop):
    test = whole[int(feature * prop):,:]
    return np.squeeze(np.asarray(test[:, dimension]))

def main():
    whole = data
    #prepare training set and testing set
    np.random.shuffle(whole)
    
    print("running bagging decision tree...with proportion of training", prop)
    #errors
    np.random.shuffle(whole)
    clf = BaggingClassifier(DecisionTreeClassifier(max_depth = 14))
    clf.fit(X = get_train_X(whole, prop), y = get_train_y(whole, prop))
    
    acc = clf.score(get_train_X(whole, prop), get_train_y(whole, prop))
    print ("training accuracy ", acc * 100, "%")
    
    acc = clf.score(get_test_X(whole, prop), get_test_y(whole, prop))
    print ("testing accuracy ", acc * 100, "%")
    
    acc = cross_val_score(clf, X, y, cv = 10).mean()
    print ("validation accuracy ", acc * 100, "%")
    
main()

running bagging decision tree...with proportion of training 0.4
training accuracy  99.6111609623 %
testing accuracy  98.5045291403 %
validation accuracy  98.7301233309 %
